In [1]:
import networkx as nx
import matplotlib as plt
import numpy as np
import pandas as pd
from itertools import permutations

In [2]:
def randomWave(wavelist):
    random_seed=np.random.randint(len(wavelist))
    return wavelist[random_seed]

In [3]:
def zero_runs(a):
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 0).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges      

def pathBand(path):
    path_len=len(path)
    #print(path)
    #print(path[0][0])
    resource=Resource[path[0][0]][path[0][1]][path[0][2]]
    for i in range(1,path_len):
        resource=list(map(lambda x: x[0]+x[1], zip(resource, Resource[path[i][0]][path[i][1]][path[i][2]])))
    pathbandList=zero_runs(resource) 
    
    return pathbandList

In [4]:
def bypassType(path):
    path_len=len(path)
    if(path_len==1):
        bypass_type=0
    elif(path_len==2):
        edgeLoc0=edges_list.index((path[0][0],path[0][1]))
        edgeLoc1=edges_list.index((path[1][0],path[1][1]))
        if(path[1] in Out_bypassEdges[edgeLoc0][path[0][2]] and path[0] in In_bypassEdges[edgeLoc1][path[1][2]] ) :
            bypass_type=2
        else:
            bypass_type=0
            
    elif(path_len==3):       
        edgeLoc0=edges_list.index((path[0][0],path[0][1]))
        edgeLoc1=edges_list.index((path[1][0],path[1][1]))
        edgeLoc2=edges_list.index((path[2][0],path[2][1]))
        if(path[1] in Out_bypassEdges[edgeLoc0][path[0][2]] and path[0] in In_bypassEdges[edgeLoc1][path[1][2]] ) :
            bypass_type=1
        elif(path[2] in Out_bypassEdges[edgeLoc1][path[1][2]] and path[1] in In_bypassEdges[edgeLoc2][path[2][2]] ):
            bypass_type=1
        else:
            bypass_type=0
                       
            
    return bypass_type

In [5]:
def pathBand_Ocuupy(loc1,loc2,wave1,band,index):
    path=SD_pathLink[loc1][loc2]
    pathBand=ALL_SD_bandwidth[loc1][loc2]
    for i in range(len(path)):
        for j in range(band):
            #print(path)
            #print(wave1)
            #print(Resource[path[i][0]][path[i][1]][path[i][2]][wave1+j])
            Resource[path[i][0]][path[i][1]][path[i][2]][wave1+j]+=index+1
            
    return

        
def pathUpdate():
    SR_bandwidth=[]
    #MaxBand=[]
    Hops=[]
    for i in range(len(SD_pathLink)):
        band=[]
        #Max=[]
        hops=[]        
        for j in range(len(SD_pathLink[i])):
            #cost.append(pathCost(SD_pathLink[i][j]))
            hops.append(len(SD_pathLink[i][j]))
            b=pathBand(SD_pathLink[i][j])
            #Max.append(max(b[:,1] - b[:,0]))
            band.append(b)            
        #SR_pathLink_cost.append(cost)
        SR_bandwidth.append(band)
        #MaxBand.append(Max)
        Hops.append(hops)
    global ALL_SD_bandwidth
    ALL_SD_bandwidth=SR_bandwidth
        
    return

In [6]:
def CoreSetup():
    Nodes_list=list(G.nodes)
    #Link_bypass=[]
    for i in Nodes_list:
        neighborList=list(G.neighbors(i))
        #bypasscore_num=len(neighborList)-1# set the bypass core number within each fiber 
        for j in range(len(neighborList)):
            dis=1
            while(j+dis<len(neighborList)):
                input_loc=edges_list.index((neighborList[j],i))
                output_loc=edges_list.index((i,neighborList[j+dis]))
                successFlag=0
                for m in range(7):
                    if(len(bypassEdges[input_loc][m])==0):
                        for n in range(7):
                            if(len(bypassEdges[output_loc][n])==0):
                                bypassEdges[output_loc][n].append(((neighborList[j],i,m)))
                                bypassEdges[input_loc][m].append(((i,neighborList[j+dis],n)))
                                successFlag=1
                                break
                        if(successFlag==1):
                            break
                input_loc=edges_list.index((neighborList[j+dis],i))
                output_loc=edges_list.index((i,neighborList[j]))
                successFlag=0
                for m in range(7):
                    if(len(bypassEdges[input_loc][m])==0):
                        for n in range(7):
                            if(len(bypassEdges[output_loc][n])==0):
                                bypassEdges[output_loc][n].append(((neighborList[j+dis],i,m)))
                                bypassEdges[input_loc][m].append(((i,neighborList[j],n)))
                                successFlag=1
                                break
                        if(successFlag==1):
                            break
                                                                                                        
                dis+=1
    return;

In [7]:
def edgeToLinks(path):
    pathlist=[]
    path_len=len(path)
    corePossib=7**path_len
    for i in range(corePossib):
        List=[]
        List.append((path[0]+(i%7,)))
        edgeLoc=edges_list.index(path[0])
        
        if(len(In_bypassEdges[edgeLoc][i%7])>0):
            List=[]
            continue
        for j in range(1,path_len):
            List.append((path[j]+((int(i/(7**j)%7)),)))
            edgeLoc0=edges_list.index(path[j-1])
            edgeLoc1=edges_list.index(path[j])
            if(len(Out_bypassEdges[edgeLoc0][List[j-1][2]])>0 and List[j] not in Out_bypassEdges[edgeLoc0][List[j-1][2]] ) :

                List=[]
                break
            if(len(In_bypassEdges[edgeLoc1][List[j][2]])>0 and List[j-1] not in In_bypassEdges[edgeLoc1][List[j][2]] ) :

                List=[]
                break
        edgeLoc=edges_list.index(path[path_len-1])
        if(len(Out_bypassEdges[edgeLoc][int(i/(7**(path_len-1)))])>0):            
            List=[]
            continue 
        if(len(List)>0):
            pathlist.append(List)    
    return pathlist

In [8]:
def PathAlgorithm_1(PathList,hopsList,bandwidth,pathBandList,bypassList,wave,list1,list2):
    Max=np.zeros(len(pathBandList))
    Sum=np.zeros(len(pathBandList))
    for i in range(len(pathBandList)):
        #print (pathBandList[i][:,1] - pathBandList[i][:,0])
        #print(i)
        #print(PathList[0])
        Max[i]=max(pathBandList[i][:,1] - pathBandList[i][:,0])
        Sum[i]=sum(pathBandList[i][:,1] - pathBandList[i][:,0])
    
    infor={"place":range(len(PathList)),
           "bypass":bypassList,
           "bandwidth": Max,
           "hops": hopsList,
           "sum":Sum
         }
    
    Path_infor=pd.DataFrame(infor)     
    #print(Path_infor)
    df=Path_infor.sort_values(by=list1,ascending=list2)  
    df.reset_index(drop=True, inplace=True)#reset the index sequence
    #print(df)
    Path=[]
    successful_flag=0
    for i in range(len(df)):
        if(df["bandwidth"][i]>bandwidth):
            for j in pathBandList[df["place"][i]]:
                if(wave>=j[0] and wave+bandwidth<=j[1]):
                    Path=PathList[df["place"][i]]
                    traffic_feedback.append(df["bypass"][i])
                    successful_flag=1
                    break 
        if(successful_flag==1):
            break
    if(len(Path)==0):
        traffic_feedback.append(-1)
        #print(df)
    return Path

In [9]:
def PathOccupy_percent():
    Bypass_2=[]
    Bypass_1=[]
    Bypass_0=[]
    for i in range(len(edges_list)):
        if(edges_list[i] in Traffic_SD_pairs):
            for j in range(len((SD_pathLink[i]))):
                percent=1-(sum(ALL_SD_bandwidth[i][j][:,1]-ALL_SD_bandwidth[i][j][:,0])-1)/160
                if(Bypass_Type[i][j]==2):
                    Bypass_2.append((percent))
                elif(Bypass_Type[i][j]==1):
                    Bypass_1.append((percent))
                elif(Bypass_Type[i][j]==0):
                    Bypass_0.append((percent))
    Bypass=[]
    Bypass.append(Bypass_0)
    Bypass.append(Bypass_1)
    
    Bypass.append(Bypass_2)
    
    return Bypass

In [10]:
def Link_percentage():
    Edge=[]
    Link=[]
    for i in edges_list:
        Edge.append(str(i))
    for j in range(7):        
        occupy=[]
        for i in edges_list:
            #print(i)
            #print(list(np.nonzero(Resource[i[0]][i[1]][j])))
            k=np.array(Resource[i[0]][i[1]][j])
            per=np.count_nonzero(k)/160
            occupy.append(per)
        Link.append(occupy)
    df=pd.DataFrame(Link,columns=Edge)
    #print(df)
    #df.rename()
    return df
        

In [11]:
def Path_percent():
    Bypass_2=[]
    Bypass_1=[]
    Bypass_0=[]
    for i in range(len(edges_list)):
        if(edges_list[i] in Traffic_SD_pairs):
            for j in range(len((SD_pathLink[i]))):
                #percent=1-(sum(ALL_SD_bandwidth[i][j][:,1]-ALL_SD_bandwidth[i][j][:,0])-1)/160
                if(Bypass_Type[i][j]==2):
                    Bypass_2.append((SD_pathLink[i][j]))
                elif(Bypass_Type[i][j]==1):
                    Bypass_1.append((SD_pathLink[i][j]))
                elif(Bypass_Type[i][j]==0):
                    Bypass_0.append((SD_pathLink[i][j]))
    Bypass=[]
    Bypass.append(Bypass_0)
    Bypass.append(Bypass_1)
    
    Bypass.append(Bypass_2)
    
    return Bypass

In [12]:
TrafficCost=0
node_list=[0,1,2,3]
Traffic_SD_pairs=[(0,3),(0,1),(0,2),(1,2),(2,3),(1,3),(2,1)]
SR_pair=[(0,1),(1,0),(1,2),(2,1),(0,2),(2,0),(2,3),(3,2),(0,3),(3,0),(1,3),(3,1)]
edges_list=[(0,1),(1,0),(1,2),(2,1),(0,2),(2,0),(2,3),(3,2),(1,3),(3,1)]
bypassEdges=[]
In_bypassEdges=[]
Out_bypassEdges=[]
for i in range(len(edges_list)):
    BY=[]
    BY0=[]
    BY1=[]
    
    for j in range(7):
        BY.append([])
        BY0.append([])
        BY1.append([])
    bypassEdges.append(BY)
    In_bypassEdges.append(BY0)
    Out_bypassEdges.append(BY1)
    

   
#先对D=2的节点进行bypass设置，在对D=3的节点进行bypass设置
#每次设置都在该link中不存在

G=nx.DiGraph()
G.add_nodes_from(node_list)
G.add_edges_from(edges_list)
nx.draw(G)

#plt.show()


link_list=[] 

for k in range(len(edges_list)):
    link_list0=[]
    for i in range(7):
        link_list0.append(edges_list[k]+(i,)) 
    link_list.append(link_list0)

SD_path=[]
SR_pathLink=[]
for i in range(len(SR_pair)):
    path0=[]
    for path in nx.all_simple_paths(G, source=SR_pair[i][0], target=SR_pair[i][1]):
        path_edges=[]            
        for j in range(len(path)-1):
            path_edges.append((path[j],path[j+1]))
        path0.append(path_edges)
    SD_path.append(path0)


CoreSetup() 
for i in range(len(bypassEdges)):
    Link=bypassEdges[i]
    for j in range(7):
        if(len(Link[j])>0):
            if(Link[j][0][0]==edges_list[i][1]):
                
                Out_bypassEdges[i][j]=Link[j]
            elif(Link[j][0][1]==edges_list[i][0]):
                In_bypassEdges[i][j]=Link[j]
                

In [13]:
SD_pathLink=[]  
for i in range(len(SD_path)):
    Links=[]
    for j in range(len(SD_path[i])):
        Links=Links+edgeToLinks(SD_path[i][j])
        
    SD_pathLink.append(Links)


In [14]:
Tag_range = list(permutations(range(3), 3))
Tag_list=["hops","bypass","sum"]
FalseorTrue=[]
for i in range(8):
    k1=int(i%2)
    k2=int((i%4)/2)
    k3=int(i/4)
    FalseorTrue.append((k1,k2,k3))
    
print(FalseorTrue)

FalseTrueList=[False,True]


[(0, 0, 0), (1, 0, 0), (0, 1, 0), (1, 1, 0), (0, 0, 1), (1, 0, 1), (0, 1, 1), (1, 1, 1)]


In [15]:
Traffic=range(1,26)
traffic=[x*200 for x in Traffic]
Traffic_number=[1]+traffic
Success_list=[]
Tag_List=[]
True_List=[]
Traffic_length=[]
Bandwidth_list=[]
Link_status=[]
for tag_selection in range(len(Traffic_number)):
    for true_selection in range(1):
        print(tag_selection)
    
        Traffic_num=Traffic_number[tag_selection]
        filename='traffic_'+str(Traffic_num)+'.csv'
        #Tag=Tag_range[tag_selection]
        #choose=FalseorTrue[true_selection]
        list1=["hops","bypass","sum"]
        list2=[True,False,False]
        #for i in range(len(Tag)):
            #list1.append(Tag_list[Tag[i]])
        #for i in range(len(choose)):
            #list2.append(FalseTrueList[choose[i]])
        
        Total_bandwidth=161
        Resource=np.zeros([4,4,7,161]).tolist()
        ALL_SD_bandwidth=[]
        MaxBand=[]
        Hops=[]
        Bypass_Type=[]
        for i in range(len(SD_pathLink)):
            Band=[]
            Max=[]
            Hhops=[]
            Bypass=[]
            for j in range(len(SD_pathLink[i])):
                Hhops.append(len(SD_pathLink[i][j]))
                b=pathBand(SD_pathLink[i][j])
                Max.append(max(b[:,1] - b[:,0]))
                Band.append(b)
                Bypass.append(bypassType(SD_pathLink[i][j]))
            ALL_SD_bandwidth.append(Band)
            MaxBand.append(Max)
            Hops.append(Hhops)
            Bypass_Type.append(Bypass)
        

        
        
        # In[214]:
        
        
        Traffic=pd.read_csv(filename,sep=',',names=['S','D','band'])
        SuccessFlag=np.zeros(len(Traffic))
        BypassFlag=np.zeros(len(Traffic))
        traffic_wave=[]
        traffic_path=[]
        traffic_feedback=[]
        Path_band_usage=[]
        Traffic_num=len(Traffic)
        for i in range(Traffic_num):
            traffic=(Traffic['S'][i],Traffic['D'][i])
            band=Traffic['band'][i]
            traffic_Path=SD_pathLink[SR_pair.index(traffic)]
            traffic_Hops=Hops[SR_pair.index(traffic)]
            traffic_Band=ALL_SD_bandwidth[SR_pair.index(traffic)]
            traffic_Bypass=Bypass_Type[SR_pair.index(traffic)]
            wave=list(range(Total_bandwidth-1))
            while(len(wave)):
                PathFlag=0
                select_wave=randomWave(wave)
                #print(select_wave)
                selectPath=PathAlgorithm_1(traffic_Path,traffic_Hops,band,traffic_Band,traffic_Bypass,select_wave,list1,list2)
                if(len(selectPath) and select_wave+band<=160):
                    Pathloc=traffic_Path.index(selectPath)
                    pathBand_Ocuupy(SR_pair.index(traffic),Pathloc,select_wave,band,i)
                    PathFlag=1
                    traffic_wave.append(select_wave)
                    traffic_path.append(selectPath)
                    pathUpdate()
                    if(PathFlag==1):
                        SuccessFlag[i]=1
                    break
                else:
                    wave.remove(select_wave)

                
                
        k=0
        for i in traffic_path:
            k+=len(i)
        Traffic_length.append(k/len(traffic_path))
        k=0
        for i in range(Traffic_num):
            if(SuccessFlag[i]==1):
                k+=Traffic['band'][i]
        Bandwidth_list.append(k)
        Link_status.append(Link_percentage())        
        Success_list.append(SuccessFlag.sum()) 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


In [16]:
print(Link_status)

[   (0, 1)  (1, 0)  (1, 2)  (2, 1)  (0, 2)  (2, 0)  (2, 3)  (3, 2)   (1, 3)  \
0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0  0.00000   
1     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0  0.00000   
2     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0  0.00000   
3     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0  0.00625   
4     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0  0.00000   
5     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0  0.00000   
6     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0  0.00000   

   (3, 1)  
0     0.0  
1     0.0  
2     0.0  
3     0.0  
4     0.0  
5     0.0  
6     0.0  ,     (0, 1)   (1, 0)   (1, 2)   (2, 1)   (0, 2)   (2, 0)   (2, 3)   (3, 2)  \
0  0.00000  0.10000  0.00000  0.00000  0.10000  0.00000  0.00000  0.00000   
1  0.00000  0.00000  0.00000  0.00000  0.00000  0.00000  0.36875  0.20625   
2  0.37500  0.19375  0.00000  0.00000  0.36875

In [17]:
c={'success':Success_list,
   'length': Traffic_length,
   'band':Bandwidth_list
    
}

test=pd.DataFrame(c)
test.to_csv('Randomwavelength.csv')

In [18]:
print(type(Resource))

<class 'list'>


In [56]:
print(Link_percentage())

    (0, 1)   (1, 0)   (1, 2)   (2, 1)   (0, 2)   (2, 0)   (2, 3)   (3, 2)  \
0  0.95000  0.96250  0.97500  0.93125  0.96250  0.95000  0.93750  0.97500   
1  0.97500  0.93125  0.96250  0.98750  0.93125  0.97500  0.97500  0.90625   
2  0.88750  0.98750  0.97500  0.93125  0.97500  0.90625  0.92500  0.96875   
3  0.96875  0.95000  0.93750  0.97500  0.93125  0.96875  0.96250  0.97500   
4  0.97500  0.95000  0.91250  0.98750  0.90625  0.97500  0.95000  0.95625   
5  0.96250  0.90625  0.94375  0.96250  0.91250  0.95000  0.96875  0.94375   
6  0.92500  0.93125  0.95625  0.92500  0.95000  0.95625  0.98125  0.92500   

    (1, 3)   (3, 1)  
0  0.88750  0.98750  
1  0.98750  0.96250  
2  0.96875  0.92500  
3  0.90000  0.96875  
4  0.96250  0.95625  
5  0.91250  0.94375  
6  0.98125  0.92500  


In [57]:
print(Success_list)

[1.0, 200.0, 400.0, 600.0, 787.0, 957.0, 1127.0, 1263.0, 1373.0, 1509.0, 1653.0, 1727.0, 1805.0, 1958.0, 1981.0, 2044.0, 2146.0, 2159.0, 2081.0, 2256.0, 2250.0, 2270.0, 2303.0, 2353.0, 2252.0, 2255.0]


In [58]:
 
m=0
n=0
p=0
q=0
for i in range(Traffic_num):
    if(SuccessFlag[i]==1):
        print(str(i)+' : '+str(Traffic['band'][i]))
        if(Traffic['band'][i]==1):
            m+=1
        elif(Traffic['band'][i]==2):
            n+=1
        elif(Traffic['band'][i]==4):
            p+=1 
        elif(Traffic['band'][i]==16):
            q+=1
            

0 : 16
1 : 16
2 : 1
3 : 16
4 : 4
5 : 16
6 : 2
7 : 4
8 : 2
9 : 2
10 : 1
11 : 1
12 : 4
13 : 16
14 : 2
15 : 16
16 : 1
17 : 4
18 : 1
19 : 1
20 : 1
21 : 4
22 : 4
23 : 4
24 : 16
25 : 2
26 : 1
27 : 16
28 : 16
29 : 1
30 : 2
31 : 1
32 : 1
33 : 4
34 : 16
35 : 4
36 : 1
37 : 4
38 : 2
39 : 1
40 : 1
41 : 2
42 : 16
43 : 1
44 : 2
45 : 2
46 : 16
47 : 16
48 : 4
49 : 1
50 : 4
51 : 16
52 : 16
53 : 1
54 : 2
55 : 1
56 : 2
57 : 1
58 : 4
59 : 1
60 : 4
61 : 1
62 : 1
63 : 2
64 : 4
65 : 4
66 : 2
67 : 2
68 : 1
69 : 4
70 : 1
71 : 16
72 : 16
73 : 16
74 : 16
75 : 16
76 : 2
77 : 16
78 : 1
79 : 1
80 : 4
81 : 1
82 : 16
83 : 2
84 : 16
85 : 16
86 : 2
87 : 4
88 : 4
89 : 2
90 : 16
91 : 1
92 : 1
93 : 16
94 : 4
95 : 16
96 : 4
97 : 1
98 : 1
99 : 16
100 : 16
101 : 4
102 : 4
103 : 2
104 : 4
105 : 4
106 : 4
107 : 1
108 : 2
109 : 4
110 : 16
111 : 4
112 : 1
113 : 2
114 : 16
115 : 2
116 : 4
117 : 1
118 : 1
119 : 4
120 : 1
121 : 2
122 : 2
123 : 4
124 : 16
125 : 16
126 : 16
127 : 2
128 : 16
129 : 4
130 : 16
131 : 1
132 : 2
133 : 2
13

1133 : 2
1134 : 4
1135 : 1
1137 : 4
1138 : 2
1139 : 1
1140 : 1
1141 : 2
1142 : 4
1144 : 2
1145 : 4
1147 : 2
1148 : 1
1149 : 2
1150 : 2
1151 : 1
1152 : 4
1153 : 2
1154 : 4
1155 : 4
1156 : 1
1157 : 2
1158 : 2
1159 : 2
1160 : 4
1161 : 1
1163 : 4
1164 : 4
1165 : 2
1168 : 2
1171 : 4
1173 : 4
1174 : 2
1175 : 2
1176 : 1
1178 : 2
1179 : 1
1180 : 1
1182 : 1
1183 : 1
1184 : 1
1185 : 2
1187 : 2
1188 : 1
1189 : 4
1190 : 2
1191 : 1
1192 : 1
1193 : 1
1194 : 2
1196 : 4
1197 : 2
1198 : 1
1199 : 2
1200 : 4
1201 : 2
1202 : 4
1203 : 1
1204 : 1
1205 : 2
1207 : 4
1208 : 1
1209 : 1
1210 : 2
1212 : 2
1213 : 2
1215 : 2
1217 : 2
1219 : 1
1220 : 4
1221 : 2
1223 : 1
1224 : 1
1225 : 1
1227 : 2
1228 : 1
1230 : 4
1231 : 4
1232 : 1
1234 : 4
1235 : 2
1236 : 4
1238 : 2
1239 : 2
1241 : 1
1242 : 4
1243 : 1
1244 : 4
1245 : 4
1246 : 4
1247 : 2
1250 : 2
1252 : 1
1253 : 4
1254 : 2
1255 : 2
1256 : 2
1257 : 1
1261 : 1
1262 : 4
1263 : 1
1266 : 4
1267 : 1
1268 : 4
1271 : 2
1272 : 2
1273 : 4
1274 : 4
1277 : 2
1279 : 4
1280 : 1
1

In [59]:
print(m)
print(n)
print(p)
print(q)

960
645
445
205


In [51]:
print(Bandwidth_list)

[2, 1202, 2316, 3459, 4006, 4689, 4783, 5172, 5277, 5452, 5570, 5692, 5763, 5747, 5705, 5733, 5860, 5884, 5842, 5886, 5858, 5735, 5836, 5817, 5928, 5874]


In [ ]:
[2, 1202, 2316, 3475, 4118, 4753, 4907, 5228, 5275, 5526, 5602, 5696, 5702, 5729, 5609, 5749, 5804, 5802, 5694, 5780, 5838, 5889, 5865, 5764, 5720, 5920]